# Assessing the Model Performance:

This notebook retrieves the best model along with its corresponding best set of hyperparameters that were trained in the 04-Models notebook. The purpose of this step is to assess the performance of the model using unseen data.

By applying the best model to this new data, it is possible to evaluate how well it generalizes to examples it hasn't encountered during training. This assessment will provide insights into the model's ability to make accurate predictions and handle real-world scenarios.

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pickle

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import ConfusionMatrixDisplay, classification_report, RocCurveDisplay
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [9]:
from functions import credit_approval_data_cleaner, n_mo_delinquency

In [3]:
# import warnings
# warnings.filterwarnings("ignore")

In [10]:
applications_test = pd.read_csv('../data/applications_test.csv')
credits_test = pd.read_csv('../data/credits_test.csv')

In [14]:
test = credit_approval_data_cleaner(credits_test, applications_test, [6, 12])

In [15]:
test.head()

,id,is_delinquent,length_of_credit,number_of_delinquent_months,average_delinquency_rate,6mo_delinquency,12mo_delinquency,flag_own_car,flag_own_realty,cnt_children,...,name_family_status,name_housing_type,flag_mobil,flag_work_phone,flag_phone,flag_email,occupation_type,cnt_fam_members,age,years_employed
0,5008834,0,45,5,0.111111,0,0,N,Y,1,...,Single / not married,House / apartment,1,0,0,0,missing,2.0,30,4
1,5008839,0,14,10,0.714286,1,1,N,Y,1,...,Married,House / apartment,1,0,0,0,Managers,3.0,32,5
2,5008861,1,4,3,0.750000,1,1,Y,Y,2,...,Married,House / apartment,1,0,0,0,Laborers,4.0,43,8
3,5008891,0,8,1,0.125000,0,1,N,Y,0,...,Single / not married,Rented apartment,1,0,0,0,Laborers,1.0,42,8
4,5008893,0,37,0,0.000000,0,0,N,Y,0,...,Single / not married,Rented apartment,1,0,0,0,Laborers,1.0,42,8


In [16]:
test.shape

(5397, 23)

### Importing and Setting up the Model

In [19]:
X = test.drop(columns=[
    'id', 'is_delinquent', 'number_of_delinquent_months', 'average_delinquency_rate', 
    '6mo_delinquency', '12mo_delinquency'])

y_1 = test['6mo_delinquency']
y_2 = test['12mo_delinquency']

In [7]:
with open('../data/best_model.pkl', 'rb') as f:
    best_model = pickle.load(f)

In [8]:
best_model['model'].best_estimator_

Pipeline(steps=[('vec',
                 TfidfVectorizer(max_df=0.875, max_features=6250, min_df=6,
                                 preprocessor=<function stem_text at 0x7fee81bc7370>,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...])),
                ('model', SVC(degree=4, probability=True))])

In [9]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

Before setting up the model, it is necessary to configure its hyperparameters. These hyperparameters are stored in `best_model['model'].best_params_`. The next two code cells demonstrate the process of preparing the hyperparameters for the model implementation.

## Model Evaluation

#### Importing and cleaning test data

In [20]:
y_1.value_counts(normalize=True)

0    0.53363
1    0.46637
Name: 6mo_delinquency, dtype: float64

In [19]:
print('Test set Accuracy:', bm.score(X, y_1))
print(f"Test set AUC:", roc_auc_score(y_1, bm.predict_proba(X)[:, 1]))


print('\n')
print(classification_report(y_test, bm.predict(X_test)))
print('\n')

Training set Accuracy: 0.9791754756871036
Test set Accuracy: 0.9221987315010571
Test set AUC: 0.9663539349734281


              precision    recall  f1-score   support

           0       0.90      0.89      0.89       881
           1       0.94      0.94      0.94      1484

    accuracy                           0.92      2365
   macro avg       0.92      0.92      0.92      2365
weighted avg       0.92      0.92      0.92      2365





In [3]:
fig, ax = plt.subplots(1, 2, figsize=(13, 6))
ConfusionMatrixDisplay.from_estimator(bm, X, y_1, cmap='Blues', colorbar=False, ax=ax[0])
RocCurveDisplay.from_estimator(bm, X, y_1, ax=ax[1])

ax[0].set_title('Confusion Matrix')
ax[1].set_title('ROC Curve')

fig.suptitle('Model Performance', fontsize=18)

plt.savefig('../Images/model_performance.png', dpi=300);

### Conclusion: 
The model's exceptional performance, surpassing the benchmark by approximately 30 percentage points and achieving a remarkable 97% AUC score, is truly noteworthy considering the limitations of the available data. With an overall accuracy of 92%, a recall rate of 94%, and a specificity of 89%, there is already a strong foundation for learning and improvement.

It is reasonable to expect that with a larger and more consistent dataset, one that relies solely on the most up-to-date data, the model has the potential to further outperform its current performance. By incorporating a larger dataset, the model can capture more diverse patterns and gain a better understanding of the underlying relationships in the data. This can lead to even more accurate predictions and higher performance metrics.